In [1]:
import pandas as pd
#import essay
import numpy as np
import pickle
import re

In [2]:
df_essays = pd.read_csv('essays.csv', encoding='cp1252', delimiter=',', quotechar='"')

# for every essay, we replace the personalitiy categories 
# of the essay wich are "y" and "n" with "1" and "0" 
for e in df_essays.columns[2:7]:
    df_essays[e] = df_essays[e].replace('n', '0')
    df_essays[e] = df_essays[e].replace('y', '1')
    # not sure if we need this line: furter investigation possible:
    df_essays[e] = pd.to_numeric(df_essays[e])

df_essays = df_essays[["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_essays.head()

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,I can't believe it! It's really happening! M...,1,0,1,1,0
4,"Well, here I go with the good old stream of co...",1,0,1,0,1


In [7]:
df_kaggle = pd.read_csv('data/training/mbti_1.csv',  skiprows=0 )
df_kaggle

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [8]:
def mbti_to_big5(mbti):
    mbti = mbti.lower()
    cEXT, cNEU, cAGR, cCON, cOPN = 0,np.NaN,0,0,0
    
    ## IN MBTI, extrovert or introvert
    ## correlates with Extroversion
    if mbti[0] == "i":
        cEXT = 0
    elif mbti[0] == "e":
        cEXT = 1
        
    ## IN MBTI, Feeler or Thinker
    ## correlates with Agrreableness
    if mbti[2] == "t":
        cAGR = 0
    elif mbti[2] == "f":
        cAGR = 1

    ## IN MBTI, Judger or Perceiver
    ## correlates with Conscientiousness
    if mbti[3] == "p":
        cCON = 0
    elif mbti[3] == "j":
        cCON = 1
        
    ## IN MBTI, Intuition or Sensing 
    ## correlates with Openness
    if mbti[1] == "n":
        cOPN = 1
    elif mbti[1] == "s":
        cOPN = 0   
        
    return cEXT, cNEU, cAGR, cCON, cOPN

In [9]:
# simply put every row of our read dataframe into a list of 
# the object "Essay"
# remove data from list substract
def create_essays(df, subtract=None):
    essays = []
    for index, row in df.iterrows():
        essays.append(essay.Essay(row.TEXT, row.cEXT, row.cNEU, row.cAGR, row.cCON, row.cOPN))  

    # remove scentences which do not contain emotionally charged words 
    # from the emotional lexicon
    if subtract != None:
        for x in essays:
            x.filtered_text = remove_unemotional_scentences(emotional_words, x.clean_text)

    return essays

def remove_unemotional_scentences(emotional_words, text_as_one_string):
    reduced_s = ""
    scentences = re.split('(?<=[.!?]) +', text_as_one_string)
    for s in scentences:
        if any(e in s for e in emotional_words):
            reduced_s = reduced_s + s + " "
        else:
            pass
    return reduced_s

In [10]:
mbti_to_big5(df_kaggle['type'][5])

(0, nan, 0, 1, 1)

In [11]:
df_kaggle["cEXT"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[0], 1)
df_kaggle["cNEU"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[1], 1)
df_kaggle["cAGR"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[2], 1)
df_kaggle["cCON"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[3], 1)
df_kaggle["cOPN"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[4], 1)

In [12]:
df_kaggle

,type,posts,cEXT,cNEU,cAGR,cCON,cOPN
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,0,NaN,1,1,1
1,ENTP,'I'm finding the lack of me in these posts ver...,1,NaN,0,0,1
2,INTP,'Good one _____ https://www.youtube.com/wat...,0,NaN,0,0,1
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",0,NaN,0,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,1,NaN,0,1,1
...,...,...,...,...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,0,NaN,1,0,0
8671,ENFP,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,INTP,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,INFP,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


In [13]:
df_kaggle = df_kaggle[["posts", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_kaggle.columns = ["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]
# remove som fancy ||| things
df_kaggle["TEXT"] = df_kaggle.apply(lambda x: x.TEXT.replace("|||", " ")[:], 1)

df_kaggle.head()

C:\Users\Piyush\AppData\Local\Temp/ipykernel_12740/4078397535.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kaggle["TEXT"] = df_kaggle.apply(lambda x: x.TEXT.replace("|||", " ")[:], 1)


,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,'http://www.youtube.com/watch?v=qsXHcwe3krw ht...,0,NaN,1,1,1
1,'I'm finding the lack of me in these posts ver...,1,NaN,0,0,1
2,'Good one _____ https://www.youtube.com/wat...,0,NaN,0,0,1
3,"'Dear INTP, I enjoyed our conversation the o...",0,NaN,0,1,1
4,'You're fired. That's another silly misconcept...,1,NaN,0,1,1


### Create data frame MBTI AND BIG5

In [15]:
# concatinate the dataframes:
frames  = [df_essays, df_kaggle]
essays_kaggle = pd.concat(frames, sort=False)
essays_kaggle.reset_index(drop=True)

# preprocess data by converting into OBJECT essay and save with pickle and removing non emotional scentences
#essays_kaggle = create_essays(essays_kaggle, emotional_words)
#pickle.dump(essays_kaggle, open("essays/essays11142.p", "wb"))
print("saved entries: ", len(essays_kaggle))

saved entries:  11142


In [16]:
essays_kaggle

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1.0,1,0,1
1,"Well, here we go with the stream of consciousn...",0,0.0,1,0,0
2,An open keyboard and buttons to push. The thin...,0,1.0,0,1,1
3,I can't believe it! It's really happening! M...,1,0.0,1,1,0
4,"Well, here I go with the good old stream of co...",1,0.0,1,0,1
...,...,...,...,...,...,...
8670,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,NaN,1,0,0
8671,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


In [18]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

stop_words = stopwords.words("english")

def preprocess(text):
    corpus = []
    
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>2 and token not in stop_words:
            corpus.append(token)
    return corpus

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Piyush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Piyush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
essays_kaggle['clean'] = essays_kaggle['TEXT'].apply(preprocess)
essays_kaggle['clean_text'] = essays_kaggle['clean'].apply(lambda x:" ".join(x))
essays_kaggle.head()

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN,clean,clean_text
0,"Well, right now I just woke up from a mid-day ...",0,1.0,1,0,1,"[right, woke, mid, day, nap, sort, weird, move...",right woke mid day nap sort weird moved texas ...
1,"Well, here we go with the stream of consciousn...",0,0.0,1,0,0,"[stream, consciousness, essay, things, like, h...",stream consciousness essay things like high sc...
2,An open keyboard and buttons to push. The thin...,0,1.0,0,1,1,"[open, keyboard, buttons, push, thing, finally...",open keyboard buttons push thing finally worke...
3,I can't believe it! It's really happening! M...,1,0.0,1,1,0,"[believe, happening, pulse, racing, like, mad,...",believe happening pulse racing like mad like f...
4,"Well, here I go with the good old stream of co...",1,0.0,1,0,1,"[good, old, stream, consciousness, assignment,...",good old stream consciousness assignment feel ...


In [20]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(essays_kaggle, test_size=0.20, random_state=42)

In [21]:
train_x = training.clean_text

train_y_cEXT = training['cEXT']
train_y_cNEU = training['cNEU']
train_y_cAGR = training['cAGR']
train_y_cCON = training['cCON']
train_y_cOPN = training['cOPN']


test_x = test.clean_text

test_y_cEXT = test['cEXT']
test_y_cNEU = test['cNEU']
test_y_cAGR = test['cAGR']
test_y_cCON = test['cCON']
test_y_cOPN = test['cOPN']

In [22]:
## BAG OF WORDS

from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer()

# create vectors from our words
train_x_vectors = bow_vectorizer.fit_transform(train_x)
test_x_vectors = bow_vectorizer.transform(test_x)
# # now that's a big thing :-O

In [23]:
## TFIDF VECTORIZER

from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer()
train_x_vectors_tf = cv.fit_transform(train_x)
test_x_vectors_tf = cv.transform(test_x)

In [45]:
x_vectors_tf = cv.fit_transform(essays_kaggle['TEXT'])



In [24]:
# for evaluation save some data for later:
evaluation = []
evaluation_tf = []
data = len(essays_kaggle)
vec_name = "MBTI"

In [53]:
from sklearn.model_selection import KFold
kfold_validation=KFold(10)

In [42]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver="newton-cg")

In [47]:
import numpy as np
from sklearn.model_selection import cross_val_score
results=cross_val_score(log_model,x_vectors_tf,essays_kaggle['cEXT'],cv=kfold_validation, scoring='accuracy')
print(results)
print(np.mean(results))

[0.55964126 0.54977578 0.77289048 0.82315978 0.81508079 0.84290844
 0.83662478 0.84201077 0.83393178 0.83213645]
0.7708160307863233


In [48]:
import numpy as np
from sklearn.model_selection import cross_val_score
results=cross_val_score(log_model,x_vectors_tf,essays_kaggle['cOPN'],cv=kfold_validation, scoring='accuracy')
print(results)
print(np.mean(results))

[0.59461883 0.62959641 0.82585278 0.86086176 0.87073609 0.86624776
 0.88958707 0.86535009 0.86355476 0.88150808]
0.8147913630837849


In [49]:
import numpy as np
from sklearn.model_selection import cross_val_score
results=cross_val_score(log_model,x_vectors_tf,essays_kaggle['cAGR'],cv=kfold_validation, scoring='accuracy')
print(results)
print(np.mean(results))

[0.55156951 0.55067265 0.77917415 0.8572711  0.84829443 0.86445242
 0.85906643 0.86355476 0.86265709 0.8438061 ]
0.788051863361538


In [50]:
import numpy as np
from sklearn.model_selection import cross_val_score
results=cross_val_score(log_model,x_vectors_tf,essays_kaggle['cCON'],cv=kfold_validation, scoring='accuracy')
print(results)
print(np.mean(results))

[0.58475336 0.56233184 0.76570916 0.79533214 0.77378815 0.80789946
 0.81149013 0.78007181 0.77917415 0.76211849]
0.7422668684738066


In [51]:
import numpy as np
from sklearn.model_selection import cross_val_score
results=cross_val_score(log_model,x_vectors_tf,essays_kaggle['cNEU'],cv=kfold_validation, scoring='accuracy')
print(results)
print(np.mean(results))

c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py", line 1344, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return

[nan nan nan nan nan nan nan nan nan nan]
nan


c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py", line 1344, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\users\piyush\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return

In [ ]:
## SVM

from sklearn import svm
name = "svm"

print("training Extraversion cEXT using SVM...")
clf_svm_cEXT = svm.SVC(kernel='linear')
clf_svm_cEXT.fit(train_x_vectors, train_y_cEXT)
evaluation.append([data, vec_name, name, "cEXT", clf_svm_cEXT.score(test_x_vectors, test_y_cEXT)])
print("cEXT score: ", clf_svm_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using SVM...")
    clf_svm_cNEU = svm.SVC(kernel='linear')
    clf_svm_cNEU.fit(train_x_vectors, train_y_cNEU)
    evaluation.append([data, vec_name, name, "cNEU", clf_svm_cNEU.score(test_x_vectors, test_y_cNEU)])
    print("cNEU score: ", clf_svm_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using SVM...")
clf_svm_cAGR = svm.SVC(kernel='linear')
clf_svm_cAGR.fit(train_x_vectors, train_y_cAGR)
evaluation.append([data, vec_name, name, "cAGR", clf_svm_cAGR.score(test_x_vectors, test_y_cAGR)])
print("cAGR score: ", clf_svm_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using SVM...")
clf_svm_cCON = svm.SVC(kernel='linear')
clf_svm_cCON.fit(train_x_vectors, train_y_cCON)
evaluation.append([data, vec_name, name, "cCON", clf_svm_cCON.score(test_x_vectors, test_y_cCON)])
print("cCON score: ", clf_svm_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using SVM...")
clf_svm_cOPN = svm.SVC(kernel='linear')
clf_svm_cOPN.fit(train_x_vectors, train_y_cOPN)
evaluation.append([data, vec_name, name, "cOPN", clf_svm_cOPN.score(test_x_vectors, test_y_cOPN)])
print("cOPN score: ", clf_svm_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using SVM...
cEXT score:  0.7231942575145806
training Neuroticism cNEU using SVM...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using SVM...
cAGR score:  0.7039030955585465
training Conscientiousness cCON using SVM...
cCON score:  0.680574248541947
training Openness to Experience cOPN using SVM...
cOPN score:  0.7747868999551368


In [ ]:
evaluation

[[11142, 'MBTI', 'svm', 'cEXT', 0.7231942575145806],
 [11142, 'MBTI', 'svm', 'cAGR', 0.7039030955585465],
 [11142, 'MBTI', 'svm', 'cCON', 0.680574248541947],
 [11142, 'MBTI', 'svm', 'cOPN', 0.7747868999551368]]

In [ ]:
### SVM - TFIDF

print("training Extraversion cEXT using SVM...")
clf_svm_cEXT = svm.SVC(kernel='linear')
clf_svm_cEXT.fit(train_x_vectors_tf, train_y_cEXT)
evaluation_tf.append([data, vec_name, name, "cEXT", clf_svm_cEXT.score(test_x_vectors_tf, test_y_cEXT)])
print("cEXT score: ", clf_svm_cEXT.score(test_x_vectors_tf, test_y_cEXT))

try:
    print("training Neuroticism cNEU using SVM...")
    clf_svm_cNEU = svm.SVC(kernel='linear')
    clf_svm_cNEU.fit(train_x_vectors_tf, train_y_cNEU)
    evaluation_tf.append([data, vec_name, name, "cNEU", clf_svm_cNEU.score(test_x_vectors_tf, test_y_cNEU)])
    print("cNEU score: ", clf_svm_cNEU.score(test_x_vectors_tf, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using SVM...")
clf_svm_cAGR = svm.SVC(kernel='linear')
clf_svm_cAGR.fit(train_x_vectors_tf, train_y_cAGR)
evaluation_tf.append([data, vec_name, name, "cAGR", clf_svm_cAGR.score(test_x_vectors_tf, test_y_cAGR)])
print("cAGR score: ", clf_svm_cAGR.score(test_x_vectors_tf, test_y_cAGR))

print("training Conscientiousness cCON using SVM...")
clf_svm_cCON = svm.SVC(kernel='linear')
clf_svm_cCON.fit(train_x_vectors_tf, train_y_cCON)
evaluation_tf.append([data, vec_name, name, "cCON", clf_svm_cCON.score(test_x_vectors_tf, test_y_cCON)])
print("cCON score: ", clf_svm_cCON.score(test_x_vectors_tf, test_y_cCON))

print("training Openness to Experience cOPN using SVM...")
clf_svm_cOPN = svm.SVC(kernel='linear')
clf_svm_cOPN.fit(train_x_vectors_tf, train_y_cOPN)
evaluation_tf.append([data, vec_name, name, "cOPN", clf_svm_cOPN.score(test_x_vectors_tf, test_y_cOPN)])
print("cOPN score: ", clf_svm_cOPN.score(test_x_vectors_tf, test_y_cOPN))

training Extraversion cEXT using SVM...
cEXT score:  0.7801704800358905
training Neuroticism cNEU using SVM...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using SVM...
cAGR score:  0.7806191117092867
training Conscientiousness cCON using SVM...
cCON score:  0.7317182593091073
training Openness to Experience cOPN using SVM...
cOPN score:  0.8218932256617317


In [ ]:
## Logistic Regression - BOW

from sklearn.linear_model import LogisticRegression
name="logR"

print("training Extraversion cEXT using Logistic Regression...")
clf_log_cEXT = LogisticRegression(solver="newton-cg")
clf_log_cEXT.fit(train_x_vectors, train_y_cEXT)
evaluation.append([data, vec_name, name, "cEXT", clf_log_cEXT.score(test_x_vectors, test_y_cEXT)])
print("cEXT score: ", clf_log_cEXT.score(test_x_vectors, test_y_cEXT))

try:
  print("training Neuroticism cNEU using Logistic Regression...")
  clf_log_cNEU = LogisticRegression(solver="newton-cg")
  clf_log_cNEU.fit(train_x_vectors, train_y_cNEU)
  evaluation.append([data, vec_name, name, "cNEU", clf_log_cNEU.score(test_x_vectors, test_y_cNEU)])
  print("cNEU score: ", clf_log_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using Logistic Regression...")
clf_log_cAGR = LogisticRegression(solver="newton-cg")
clf_log_cAGR.fit(train_x_vectors, train_y_cAGR)
evaluation.append([data, vec_name, name, "cAGR", clf_log_cAGR.score(test_x_vectors, test_y_cAGR)])
print("cAGR score: ", clf_log_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using Logistic Regression...")
clf_log_cCON = LogisticRegression(solver="newton-cg")
clf_log_cCON.fit(train_x_vectors, train_y_cCON)
evaluation.append([data, vec_name, name, "cCON", clf_log_cCON.score(test_x_vectors, test_y_cCON)])
print("cCON score: ", clf_log_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using Logistic Regression...")
clf_log_cOPN = LogisticRegression(solver="newton-cg")
clf_log_cOPN.fit(train_x_vectors, train_y_cOPN)
evaluation.append([data, vec_name, name, "cOPN", clf_log_cOPN.score(test_x_vectors, test_y_cOPN)])
print("cOPN score: ", clf_log_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using Logistic Regression...
cEXT score:  0.7590847913862718
training Neuroticism cNEU using Logistic Regression...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Logistic Regression...
cAGR score:  0.7200538358008075
training Conscientiousness cCON using Logistic Regression...
cCON score:  0.696276357110812
training Openness to Experience cOPN using Logistic Regression...
cOPN score:  0.7967698519515478


In [ ]:
## Logistic Regression - TFIDF

from sklearn.linear_model import LogisticRegression
name="logR"

print("training Extraversion cEXT using Logistic Regression...")
clf_log_cEXT = LogisticRegression(solver="newton-cg")
clf_log_cEXT.fit(train_x_vectors_tf, train_y_cEXT)
evaluation_tf.append([data, vec_name, name, "cEXT", clf_log_cEXT.score(test_x_vectors_tf, test_y_cEXT)])
print("cEXT score: ", clf_log_cEXT.score(test_x_vectors_tf, test_y_cEXT))

try:
  print("training Neuroticism cNEU using Logistic Regression...")
  clf_log_cNEU = LogisticRegression(solver="newton-cg")
  clf_log_cNEU.fit(train_x_vectors_tf, train_y_cNEU)
  evaluation_tf.append([data, vec_name, name, "cNEU", clf_log_cNEU.score(test_x_vectors_tf, test_y_cNEU)])
  print("cNEU score: ", clf_log_cNEU.score(test_x_vectors_tf, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using Logistic Regression...")
clf_log_cAGR = LogisticRegression(solver="newton-cg")
clf_log_cAGR.fit(train_x_vectors_tf, train_y_cAGR)
evaluation_tf.append([data, vec_name, name, "cAGR", clf_log_cAGR.score(test_x_vectors_tf, test_y_cAGR)])
print("cAGR score: ", clf_log_cAGR.score(test_x_vectors_tf, test_y_cAGR))

print("training Conscientiousness cCON using Logistic Regression...")
clf_log_cCON = LogisticRegression(solver="newton-cg")
clf_log_cCON.fit(train_x_vectors_tf, train_y_cCON)
evaluation_tf.append([data, vec_name, name, "cCON", clf_log_cCON.score(test_x_vectors_tf, test_y_cCON)])
print("cCON score: ", clf_log_cCON.score(test_x_vectors_tf, test_y_cCON))

print("training Openness to Experience cOPN using Logistic Regression...")
clf_log_cOPN = LogisticRegression(solver="newton-cg")
clf_log_cOPN.fit(train_x_vectors_tf, train_y_cOPN)
evaluation_tf.append([data, vec_name, name, "cOPN", clf_log_cOPN.score(test_x_vectors_tf, test_y_cOPN)])
print("cOPN score: ", clf_log_cOPN.score(test_x_vectors_tf, test_y_cOPN))

training Extraversion cEXT using Logistic Regression...
cEXT score:  0.7765814266487214
training Neuroticism cNEU using Logistic Regression...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Logistic Regression...
cAGR score:  0.7963212202781517
training Conscientiousness cCON using Logistic Regression...
cCON score:  0.7420367877972185
training Openness to Experience cOPN using Logistic Regression...
cOPN score:  0.8156123822341858


In [ ]:
evaluation

[[11142, 'MBTI', 'svm', 'cEXT', 0.7231942575145806],
 [11142, 'MBTI', 'svm', 'cAGR', 0.7039030955585465],
 [11142, 'MBTI', 'svm', 'cCON', 0.680574248541947],
 [11142, 'MBTI', 'svm', 'cOPN', 0.7747868999551368],
 [11142, 'MBTI', 'logR', 'cEXT', 0.7590847913862718],
 [11142, 'MBTI', 'logR', 'cEXT', 0.7590847913862718],
 [11142, 'MBTI', 'logR', 'cAGR', 0.7200538358008075],
 [11142, 'MBTI', 'logR', 'cCON', 0.696276357110812],
 [11142, 'MBTI', 'logR', 'cOPN', 0.7967698519515478]]

In [ ]:
evaluation_tf

[[11142, 'MBTI', 'svm', 'cEXT', 0.7801704800358905],
 [11142, 'MBTI', 'svm', 'cAGR', 0.7806191117092867],
 [11142, 'MBTI', 'svm', 'cCON', 0.7317182593091073],
 [11142, 'MBTI', 'svm', 'cOPN', 0.8218932256617317],
 [11142, 'MBTI', 'logR', 'cEXT', 0.7765814266487214],
 [11142, 'MBTI', 'logR', 'cAGR', 0.7963212202781517],
 [11142, 'MBTI', 'logR', 'cCON', 0.7420367877972185],
 [11142, 'MBTI', 'logR', 'cOPN', 0.8156123822341858]]

In [ ]:
## Random Forest Classifier - BOW

from sklearn.ensemble import RandomForestClassifier
name="RF"


print("training Extraversion cEXT using Random Forest...")
clf_rf_cEXT = RandomForestClassifier(n_estimators=100)
clf_rf_cEXT.fit(train_x_vectors, train_y_cEXT)
evaluation.append([data, vec_name, name, "cEXT", clf_rf_cEXT.score(test_x_vectors, test_y_cEXT)])
print("cEXT score: ", clf_rf_cEXT.score(test_x_vectors, test_y_cEXT))

try:
  print("training Neuroticism cNEU using Random Forest...")
  clf_rf_cNEU = RandomForestClassifier(n_estimators=100)
  clf_rf_cNEU.fit(train_x_vectors, train_y_cNEU)
  evaluation.append([data, vec_name, name, "cNEU", clf_rf_cNEU.score(test_x_vectors, test_y_cNEU)])
  print("cNEU score: ", clf_rf_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")


print("training Agreeableness cAGR using using Random Forest...")
clf_rf_cAGR = RandomForestClassifier(n_estimators=100)
clf_rf_cAGR.fit(train_x_vectors, train_y_cAGR)
evaluation.append([data, vec_name, name, "cAGR", clf_rf_cAGR.score(test_x_vectors, test_y_cAGR)])
print("cAGR score: ", clf_rf_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using Random Forest...")
clf_rf_cCON = RandomForestClassifier(n_estimators=100)
clf_rf_cCON.fit(train_x_vectors, train_y_cCON)
evaluation.append([data, vec_name, name, "cCON", clf_rf_cCON.score(test_x_vectors, test_y_cCON)])
print("cCON score: ", clf_rf_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using Random Forest...")
clf_rf_cOPN = RandomForestClassifier(n_estimators=100)
clf_rf_cOPN.fit(train_x_vectors, train_y_cOPN)
evaluation.append([data, vec_name, name, "cOPN", clf_rf_cOPN.score(test_x_vectors, test_y_cOPN)])
print("cOPN score: ", clf_rf_cOPN.score(test_x_vectors, test_y_cOPN))


training Extraversion cEXT using Random Forest...
cEXT score:  0.7142216240466577
training Neuroticism cNEU using Random Forest...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Random Forest...
cAGR score:  0.7101839389860924
training Conscientiousness cCON using Random Forest...
cCON score:  0.5962314939434724
training Openness to Experience cOPN using Random Forest...
cOPN score:  0.7886944818304172


In [ ]:
## Random Forest Classifier - TFIDF

from sklearn.ensemble import RandomForestClassifier
name="RF"


print("training Extraversion cEXT using Random Forest...")
clf_rf_cEXT = RandomForestClassifier(n_estimators=100)
clf_rf_cEXT.fit(train_x_vectors_tf, train_y_cEXT)
evaluation_tf.append([data, vec_name, name, "cEXT", clf_rf_cEXT.score(test_x_vectors_tf, test_y_cEXT)])
print("cEXT score: ", clf_rf_cEXT.score(test_x_vectors_tf, test_y_cEXT))

try:
  print("training Neuroticism cNEU using Random Forest...")
  clf_rf_cNEU = RandomForestClassifier(n_estimators=100)
  clf_rf_cNEU.fit(train_x_vectors_tf, train_y_cNEU)
  evaluation_tf.append([data, vec_name, name, "cNEU", clf_rf_cNEU.score(test_x_vectors_tf, test_y_cNEU)])
  print("cNEU score: ", clf_rf_cNEU.score(test_x_vectors_tf, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using Random Forest...")
clf_rf_cAGR = RandomForestClassifier(n_estimators=100)
clf_rf_cAGR.fit(train_x_vectors_tf, train_y_cAGR)
evaluation_tf.append([data, vec_name, name, "cAGR", clf_rf_cAGR.score(test_x_vectors_tf, test_y_cAGR)])
print("cAGR score: ", clf_rf_cAGR.score(test_x_vectors_tf, test_y_cAGR))

print("training Conscientiousness cCON using Random Forest...")
clf_rf_cCON = RandomForestClassifier(n_estimators=100)
clf_rf_cCON.fit(train_x_vectors_tf, train_y_cCON)
evaluation_tf.append([data, vec_name, name, "cCON", clf_rf_cCON.score(test_x_vectors_tf, test_y_cCON)])
print("cCON score: ", clf_rf_cCON.score(test_x_vectors_tf, test_y_cCON))

print("training Openness to Experience cOPN using Random Forest...")
clf_rf_cOPN = RandomForestClassifier(n_estimators=100)
clf_rf_cOPN.fit(train_x_vectors_tf, train_y_cOPN)
evaluation_tf.append([data, vec_name, name, "cOPN", clf_rf_cOPN.score(test_x_vectors_tf, test_y_cOPN)])
print("cOPN score: ", clf_rf_cOPN.score(test_x_vectors_tf, test_y_cOPN))


training Extraversion cEXT using Random Forest...
cEXT score:  0.7205024674742037
training Neuroticism cNEU using Random Forest...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Random Forest...
cAGR score:  0.7074921489457155
training Conscientiousness cCON using Random Forest...
cCON score:  0.6173171825930911
training Openness to Experience cOPN using Random Forest...
cOPN score:  0.7949753252579632


In [ ]:
## Decision Tree - BOW

from sklearn import tree
name = "tree"

print("training Extraversion cEXT using dec...")
clf_dec_cEXT = tree.DecisionTreeClassifier()
clf_dec_cEXT.fit(train_x_vectors, train_y_cEXT)
evaluation.append([data, vec_name, name, "cEXT", clf_dec_cEXT.score(test_x_vectors, test_y_cEXT)])
print("cEXT score: ", clf_dec_cEXT.score(test_x_vectors, test_y_cEXT))

print("training Neuroticism cNEU using dec...")
'''clf_dec_cNEU = tree.DecisionTreeClassifier()
clf_dec_cNEU.fit(train_x_vectors, train_y_cNEU)
evaluation.append([data, vec_name, name, "cNEU", clf_dec_cNEU.score(test_x_vectors, test_y_cNEU)])
print("cNEU score: ", clf_dec_cNEU.score(test_x_vectors, test_y_cNEU))
'''
print("training Agreeableness cAGR using using dec...")
clf_dec_cAGR = tree.DecisionTreeClassifier()
clf_dec_cAGR.fit(train_x_vectors, train_y_cAGR)
evaluation.append([data, vec_name, name, "cAGR", clf_dec_cAGR.score(test_x_vectors, test_y_cAGR)])
print("cAGR score: ", clf_dec_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using dec...")
clf_dec_cCON = tree.DecisionTreeClassifier()
clf_dec_cCON.fit(train_x_vectors, train_y_cCON)
evaluation.append([data, vec_name, name, "cCON", clf_dec_cCON.score(test_x_vectors, test_y_cCON)])
print("cCON score: ", clf_dec_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using dec...")
clf_dec_cOPN = tree.DecisionTreeClassifier()
clf_dec_cOPN.fit(train_x_vectors, train_y_cOPN)
evaluation.append([data, vec_name, name, "cOPN", clf_dec_cOPN.score(test_x_vectors, test_y_cOPN)])
print("cOPN score: ", clf_dec_cOPN.score(test_x_vectors, test_y_cOPN))


training Extraversion cEXT using dec...
cEXT score:  0.724540152534769
training Neuroticism cNEU using dec...
training Agreeableness cAGR using using dec...
cAGR score:  0.6828174069089278
training Conscientiousness cCON using dec...
cCON score:  0.6460296096904441
training Openness to Experience cOPN using dec...
cOPN score:  0.7608793180798564


In [ ]:
## Decision Tree - TFIDF

from sklearn import tree
name = "tree"

print("training Extraversion cEXT using dec...")
clf_dec_cEXT = tree.DecisionTreeClassifier()
clf_dec_cEXT.fit(train_x_vectors_tf, train_y_cEXT)
evaluation_tf.append([data, vec_name, name, "cEXT", clf_dec_cEXT.score(test_x_vectors_tf, test_y_cEXT)])
print("cEXT score: ", clf_dec_cEXT.score(test_x_vectors_tf, test_y_cEXT))

print("training Neuroticism cNEU using dec...")
'''clf_dec_cNEU = tree.DecisionTreeClassifier()
clf_dec_cNEU.fit(train_x_vectors_tf, train_y_cNEU)
evaluation_tf.append([data, vec_name, name, "cNEU", clf_dec_cNEU.score(test_x_vectors_tf, test_y_cNEU)])
print("cNEU score: ", clf_dec_cNEU.score(test_x_vectors_tf, test_y_cNEU))
'''
print("training Agreeableness cAGR using using dec...")
clf_dec_cAGR = tree.DecisionTreeClassifier()
clf_dec_cAGR.fit(train_x_vectors_tf, train_y_cAGR)
evaluation_tf.append([data, vec_name, name, "cAGR", clf_dec_cAGR.score(test_x_vectors_tf, test_y_cAGR)])
print("cAGR score: ", clf_dec_cAGR.score(test_x_vectors_tf, test_y_cAGR))

print("training Conscientiousness cCON using dec...")
clf_dec_cCON = tree.DecisionTreeClassifier()
clf_dec_cCON.fit(train_x_vectors_tf, train_y_cCON)
evaluation_tf.append([data, vec_name, name, "cCON", clf_dec_cCON.score(test_x_vectors_tf, test_y_cCON)])
print("cCON score: ", clf_dec_cCON.score(test_x_vectors_tf, test_y_cCON))

print("training Openness to Experience cOPN using dec...")
clf_dec_cOPN = tree.DecisionTreeClassifier()
clf_dec_cOPN.fit(train_x_vectors_tf, train_y_cOPN)
evaluation_tf.append([data, vec_name, name, "cOPN", clf_dec_cOPN.score(test_x_vectors_tf, test_y_cOPN)])
print("cOPN score: ", clf_dec_cOPN.score(test_x_vectors_tf, test_y_cOPN))


training Extraversion cEXT using dec...
cEXT score:  0.7003140421713773
training Neuroticism cNEU using dec...
training Agreeableness cAGR using using dec...
cAGR score:  0.6680125616868551
training Conscientiousness cCON using dec...
cCON score:  0.648721399730821
training Openness to Experience cOPN using dec...
cOPN score:  0.7676087931807986


In [ ]:
train_x_vectors = train_x_vectors.toarray()
test_x_vectors = test_x_vectors.toarray()

In [ ]:
## Naive Bayes - BOW

from sklearn.naive_bayes import GaussianNB
name = "gNB"

print("training Extraversion cEXT using GaussianNaiveBayes...")
clf_gnb_cEXT = GaussianNB()
clf_gnb_cEXT.fit(train_x_vectors, train_y_cEXT)
evaluation.append([data, vec_name, name, "cEXT", clf_gnb_cEXT.score(test_x_vectors, test_y_cEXT)])
print("cEXT score: ", clf_gnb_cEXT.score(test_x_vectors, test_y_cEXT))

print("training Neuroticism cNEU using GaussianNaiveBayes...")
'''clf_gnb_cNEU = GaussianNB()
clf_gnb_cNEU.fit(train_x_vectors.toarray(), train_y_cNEU)
evaluation.append([data, vec_name, name, "cNEU", clf_gnb_cNEU.score(test_x_vectors.toarray(), test_y_cNEU)])
print("cNEU score: ", clf_gnb_cNEU.score(test_x_vectors.toarray(), test_y_cNEU))'''
    
print("training Agreeableness cAGR using using GaussianNaiveBayes...")
clf_gnb_cAGR = GaussianNB()
clf_gnb_cAGR.fit(train_x_vectors, train_y_cAGR)
evaluation.append([data, vec_name, name, "cAGR", clf_gnb_cAGR.score(test_x_vectors, test_y_cAGR)])
print("cAGR score: ", clf_gnb_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using GaussianNaiveBayes...")
clf_gnb_cCON = GaussianNB()
clf_gnb_cCON.fit(train_x_vectors, train_y_cCON)
evaluation.append([data, vec_name, name, "cCON", clf_gnb_cCON.score(test_x_vectors, test_y_cCON)])
print("cCON score: ", clf_gnb_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using GaussianNaiveBayes...")
clf_gnb_cOPN = GaussianNB()
clf_gnb_cOPN.fit(train_x_vectors, train_y_cOPN)
evaluation.append([data, vec_name, name, "cOPN", clf_gnb_cOPN.score(test_x_vectors, test_y_cOPN)])
print("cOPN score: ", clf_gnb_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using GaussianNaiveBayes...


In [ ]:
## Naive Bayes - TFIDF

from sklearn.naive_bayes import GaussianNB
name = "gNB"

print("training Extraversion cEXT using GaussianNaiveBayes...")
clf_gnb_cEXT = GaussianNB()
clf_gnb_cEXT.fit(train_x_vectors_tf.toarray(), train_y_cEXT)
evaluation_tf.append([data, vec_name, name, "cEXT", clf_gnb_cEXT.score(test_x_vectors_tf.toarray(), test_y_cEXT)])
print("cEXT score: ", clf_gnb_cEXT.score(test_x_vectors_tf.toarray(), test_y_cEXT))

print("training Neuroticism cNEU using GaussianNaiveBayes...")
'''clf_gnb_cNEU = GaussianNB()
clf_gnb_cNEU.fit(train_x_vectors_tf.toarray(), train_y_cNEU)
evaluation_tf.append([data, vec_name, name, "cNEU", clf_gnb_cNEU.score(test_x_vectors_tf.toarray(), test_y_cNEU)])
print("cNEU score: ", clf_gnb_cNEU.score(test_x_vectors_tf.toarray(), test_y_cNEU))'''
    
print("training Agreeableness cAGR using using GaussianNaiveBayes...")
clf_gnb_cAGR = GaussianNB()
clf_gnb_cAGR.fit(train_x_vectors_tf.toarray(), train_y_cAGR)
evaluation_tf.append([data, vec_name, name, "cAGR", clf_gnb_cAGR.score(test_x_vectors_tf.toarray(), test_y_cAGR)])
print("cAGR score: ", clf_gnb_cAGR.score(test_x_vectors_tf.toarray(), test_y_cAGR))

print("training Conscientiousness cCON using GaussianNaiveBayes...")
clf_gnb_cCON = GaussianNB()
clf_gnb_cCON.fit(train_x_vectors_tf.toarray(), train_y_cCON)
evaluation_tf.append([data, vec_name, name, "cCON", clf_gnb_cCON.score(test_x_vectors_tf.toarray(), test_y_cCON)])
print("cCON score: ", clf_gnb_cCON.score(test_x_vectors_tf.toarray(), test_y_cCON))

print("training Openness to Experience cOPN using GaussianNaiveBayes...")
clf_gnb_cOPN = GaussianNB()
clf_gnb_cOPN.fit(train_x_vectors_tf.toarray(), train_y_cOPN)
evaluation_tf.append([data, vec_name, name, "cOPN", clf_gnb_cOPN.score(test_x_vectors_tf.toarray(), test_y_cOPN)])
print("cOPN score: ", clf_gnb_cOPN.score(test_x_vectors_tf.toarray(), test_y_cOPN))